In [298]:
import requests, bs4, re
import pandas as pd
from datetime import date, timedelta

myDate = date(2012, 1, 3)
endDate = date(2017, 7, 28)
plusOne = timedelta(days=1)
datePattern = re.compile('\d{4}\-\D{3}\-\d{2}')
url_format = 'http://www.nasdaq.com/earnings/earnings-calendar.aspx?date=%s'
dateList = []
tickerList = []
mCapList = []

# Request Nasdaq.com Earnings Page for examination
while endDate > myDate:
    try:
        res = requests.get(url_format % myDate.strftime("%Y-%b-%d"))
    except:
        myDate += plusOne
        continue
    else:
        if myDate.weekday() not in [5,6]:
            res.raise_for_status()
            NasSoup = bs4.BeautifulSoup(res.text,"html5lib")
            
            try:
                #Save companies reporting earnings to a list
                companies = NasSoup.find_all(href=re.compile("/earnings/report"), id=True)
            except:
                myDate += plusOne
                continue
            
            # Search pattern to discover ticker symbols and market cap
            tickPattern = re.compile(r'\([A-Z]+\)')
            mCapPattern = re.compile('\d+\.\d+\D*')
            # Extract Tickers
            for ticker in companies:
                try:
                    mCapList.append(re.findall(mCapPattern,ticker.b.string)[0])
                    tickerList.append(re.findall(tickPattern,str(ticker))[-1].strip('()'))
                    dateList.append(myDate)
                except IndexError:
                    pass
    finally:
        myDate += plusOne

data = {'ReportDate': dateList,'Equities': tickerList, 'MarketCap': mCapList}
df = pd.DataFrame(data)
df.to_excel('/Users/Bert/Desktop/ZacksEarnings.xlsx', sheet_name='Sheet1', engine='xlsxwriter')

ValueError: arrays must all be same length

In [303]:
len(mCapList)

86171

In [153]:
url_formatted = 'http://www.marketwatch.com/investing/stock/%s/financials'
eqList = df.Equities.unique()
eEPS = {}

# Navigate to website for each unique ticker
for ticker in eqList:
    try:
        res = requests.get(url_formatted % ticker)
    except:
        EPS = "N/A"
        YEAR = "N/A"
        continue
    else:
        res.raise_for_status()
        epsSoup = bs4.BeautifulSoup(res.text,"html5lib")
        
        try:
            #Find the closest link with id value and go to previous sibling for earnings data
            EPS = epsSoup.find('tr', id="ratio_Eps1YrAnnualGrowth").find_previous_sibling("tr").find_all('td', class_="valueCell")
            YEAR = epsSoup.find("tr", class_="topRow").find_all('th',string=re.compile("\d{4}"))
        except:
            EPS = "N/A"
            YEAR = "N/A"
            continue
            
        # Extract 2012-2016 EPS data and save to list EarnList 4 times for each quarter's earnings
        epsPerYear = {}
        for yearEle, earnEle in zip(YEAR, EPS):
            epsPerYear[yearEle.get_text()] = earnEle.get_text()
        eEPS[ticker] = epsPerYear

In [162]:
to_merge_data = pd.DataFrame.from_dict(eEPS)

In [181]:
new_df = to_merge_data.fillna(0).T

In [184]:
new_df[new_df['2005'] != 0]

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
INSY,(1.64),(1.20),(0.39),(0.29),(0.26),0,0,0,0,0,0,0,0


In [164]:
to_merge_data.to_excel('/Users/Bert/Desktop/ProjectEarnings2.xlsx', sheet_name='Sheet1', engine='xlsxwriter')

In [215]:
Earnings = []
for ticker, date in zip(tickerList, dateList):
    if ticker in eEPS:
        try:
            year = str(date.year)
            eps = eEPS[ticker][year]
        except:
            eps = None
    else:
        eps = None
    Earnings.append(eps)

In [12]:
import pandas_datareader.data as web
import datetime

stock_tup = ()
start = datetime.datetime(2010, 1, 3)
end = datetime.datetime(2010, 1, 4)
f = web.DataReader("AAPL", 'yahoo', start, end)
stock_tup = (f['Adj Close'])

In [18]:
(stock_tup[0] - stock_tup[1])/stock_tup[0]

-0.0017289278923247999

[None,
 '0.75',
 '1.67',
 None,
 '4.44',
 None,
 None,
 '0.60',
 '4.87',
 '0.52',
 '1.73',
 None,
 None,
 '1.15',
 None,
 '0.44',
 '1.20',
 None,
 '3.83',
 '4.12',
 None,
 '()',
 None,
 None,
 '1.03',
 '1.79',
 '2.21',
 '2.24',
 '(2.31)',
 '2.75',
 '0.77',
 '1.00',
 None,
 '1.89',
 None,
 '0.26',
 '(0.06)',
 '3.58',
 '0.79',
 None,
 '(83.70)',
 '(18.00)',
 None,
 '0.75',
 '1.93',
 None,
 None,
 None,
 '(0.91)',
 None,
 '1.62',
 '0.67',
 '(1.13)',
 '2.43',
 '5.22',
 '(0.80)',
 None,
 '0.75',
 '1.71',
 '0.90',
 '1.11',
 '0.08',
 '4.18',
 '3.04',
 '2.51',
 '1.67',
 '0.39',
 None,
 '2.85',
 '0.80',
 '0.89',
 '(0.34)',
 '0.26',
 '7.57',
 '1.33',
 None,
 '0.86',
 '1.12',
 '5.04',
 '31,874.00',
 '1.06',
 '1.07',
 None,
 '3.40',
 None,
 '1.72',
 '2.70',
 '2.04',
 '2.64',
 '0.74',
 '2.02',
 '3.48',
 '1.37',
 '1.42',
 '1.22',
 '0.79',
 '14.70',
 '2.53',
 '1.59',
 '1.38',
 '2.82',
 None,
 '1.54',
 '1.78',
 '5.36',
 '2.22',
 '0.16',
 '1.93',
 '4.25',
 '6.42',
 '0.69',
 '2.85',
 '1.54',
 '2.81',
 N

In [272]:
import requests, bs4, re

url2 = 'http://www.nasdaq.com/earnings/earnings-calendar.aspx?date=2017-Aug-15'

res2 = requests.get(url2)
res2.raise_for_status()
NastySoup = bs4.BeautifulSoup(res2.text,"html5lib")
mCap = NastySoup.find_all(href=re.compile("/earnings/report"), id=True)
mCapPattern = re.compile('\d+\.\d+\D*')
for ticker in mCap:
                print(re.findall(mCapPattern,ticker.b.string)[0])

7.84B
18.82B
73.02M
13.24B
3.8B
362.36M
183.97B
8.24B
231.42M
1.11B
148.29M
11.07M
45.2B
1.99B
2.39B
35.04M
307.95M
400.35M
2.53B
1.34B
4.78B
53.16M
4.92B
156.76M
14.11B
6.82M
20.89B
1.51B
62.89M
7.13B
4.48M
456.48M
22.32B
10.73B


In [261]:
mCap

[]